<br>
<font face="Helvetica" size="5"><b>VegMapper</b></font>
<br><br>
<font face="Helvetica" size="3">License Terms

Copyright (c) 2019, California Institute of Technology ("Caltech").  U.S. Government sponsorship acknowledged.

All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
* Redistributions must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
* Neither the name of Caltech nor its operating division, the Jet Propulsion Laboratory, nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

</font>
<br>

In [1]:
#set up Python/R dual functionality for notebook
%load_ext rpy2.ipython

ModuleNotFoundError: No module named 'rpy2'

In [ ]:
%%R
#initialize R cell and load in necessary libraries
library(rgdal)
library(arm)
library(gdalUtils)
library(raster)

<br><font face="Helvetica" size="3">User inputs:</font>

In [ ]:
%%R
##### FILES #####
#all GIS data in lat lon WGS84 unless specified
#path to comma-delimited file, must have cols 'latitude', 'longitude', 'class'
in_points = "wwf_2018_clipped.csv"

#remote sensing stack in ENVI flat binary format, get stack info
in_stack =  "indo_stack_nov_2018_clipped"

#path to output comma-delimited file, same as in_points with appended remote sensing values
out_points =  "wwf_2018_clipped_testPred.csv"

#dummy corresponds to one band from the original stack, used to save your output prediction map 
in_dummy = "dummy_indo_nov_2018_clipped"

#name of the output map in ENVI flat binary format
out_pred = "op3_indo_nov_2018_clipped"

#name of output GeoTIFF
out_tif = "predSurface.tif"

#--------------------------------------------------------------#

##### PARAMETERS #####
buffer = 1

#treshold for logistic model; 
    #when probability is larger than this threshold, we say that oil palm is present
threshold = 0.5

#RUN: priors (variable order corresponds to order in which bands are read), currently Costa Rica
use_prior = TRUE
prior_mean = c(0.06491638, -26.63132179, 0.05590800, -29.64091620)
prior_scale = c(0.02038204, 7.58200324, 0.01686930, 8.73995422)
prior_mean_int = 1.99274801
prior_scale_int = 7.22600112

#lower and upper bounds for posterior credible intervals
lp = 0.025
up = 0.975

#APPLY: model parameters from your site, obtained with run_OP3-G20.R
    #the coefficients match the variable names above
    #EDIT: Fix this and connect variables from previous step
intercept =  7.77338858
posteriors = c(0.06768467, -43.16767103, 0.03948285, -9.89497112)

#--------------------------------------------------------------#

##### BAND INFO #####
bands = c(2,3,1,4)
stack_names = c("vcf", "c_rvi", "ndvi", "l_rvi_mosaic") #must be consistent with band names defined below

nodata = -9999 #NA value present in input bands: NEEDS TO BE A LIST OF NUMBERS IF NOT CONSISTENT ACROSS BANDS

#columns of the predictor variables to be used in this model (taken from pred csv)
    #column order indicated here is vcf, c_rvi, ndvi, l_rvi_mosaic, matching the priors above
index = c(13,14,15,16)
    #EDIT: find column indices using names of columns

#names for all the bands in your stack. 
bandNames = c('ndvi', 'vcf', 'c_rvi', 'lrvi', 'alpha', 'lambda', 'entropy', 'anisotropy', 'hh', 'mask', 'l_rvi_mosaic')

<hr><br>
<font face="Helvetica" size="5"><b>EXTRACT</b></font>
<br><br>
<font face="Helvetica" size="3">Read remote sensing values at training points.</font>

<br><font face="Helvetica" size="3">Create function to get intensity values from stack, execute extract routine:</font>

In [ ]:
%%R
getPixel= function(gdalObject, X, Y, buffer, ulX, ulY, cellSize, bands){
    nrow = dim(gdalObject)[1]
    ncol = dim(gdalObject)[2]
    rowOffset = ((ulY-Y)/cellSize) - buffer
    if(rowOffset<0 | (rowOffset+buffer+2) > nrow){
        return(NA)
    }
    colOffset = ((X-ulX)/cellSize) - buffer
    if(colOffset<0 | (colOffset+buffer+2) > ncol){
        return(NA)
    } 
    windowY = buffer+2
    windowX = windowY
    pixelValue = getRasterData(gdalObject, band=bands, offset=c(rowOffset, colOffset), region.dim=c(windowY, windowX))
    return(pixelValue)
}

#stack information
r_stack = stack(in_stack)
res = xres(r_stack)
r_extent = r_stack@extent
ulX = r_extent@xmin
ulY = r_extent@ymax
# s_info = GDALinfo(in_stack) also works, lacks ulY

#grab stack
gdalObj = new("GDALDataset", in_stack)

#append remote sensing information to point table and write to cvs file
inData <- read.csv(in_points, header=TRUE)
numPoints <- nrow(inData)

header <- c(colnames(inData), stack_names)
write.table(x=t(header), file=out_points, append=FALSE, col.names=FALSE, row.names=FALSE, sep=",")

print("Extracting values for...")
for(i in 1:numPoints){
    allBands <- rep(NA, length(bands))
    for (j in 1:length(bands)){
        oneBand = getPixel(gdalObj, inData$longitude[i], inData$latitude[i], buffer, ulX, ulY, res, bands[j])
        w = which (oneBand == nodata[j])
        oneBand[w]<-NA
        allBands[j] = mean(oneBand, na.rm=TRUE)
        if(i==1) print(stack_names[j])
    }
  mydata <- data.frame(t(as.vector(allBands)))
  colnames(mydata) <- stack_names
  newRow = cbind(inData[i,],mydata)
  write.table(x=newRow, file=out_points, append=TRUE, col.names=FALSE, row.names=FALSE, sep=",") 
}

<hr><br>
<font face="Helvetica" size="5"><b>RUN</b></font>
<br><br>
<font face="Helvetica" size="3">Fit Bayesian model, calculate posteriors and confusion matrix.</font>

<br><font face="Helvetica" size="3">Create model and execute prediction:</font>

In [ ]:
%%R
#impute missing values by variable means
data = read.csv(out_points)
for (i in which(sapply(data, is.numeric))) {
    for (j in which(is.na(data[, i]))) {
        data[j, i] <- mean(data[data[, "my_class"] == data[j, "my_class"], i],  na.rm = TRUE)
    }
}

#true_label: 1 for oil_palm and 0 for non oil_palm
true_label = 1*(data$my_class == 'oil_palm')

#transform interested variables into a matrix which would be used
x = as.matrix(data[, index])
all_names = names(data)
stack_names = all_names[index]
colnames(x) = stack_names

#build model by incorporating those variables
formula = as.formula(paste("true_label ~ ", paste(stack_names, collapse="+"),sep = ""))
use_data = as.data.frame(cbind(x, true_label))

#to specify prior
#if noninformative prior, use prior.mean=apply(x, 2, mean), prior.scale=Inf, prior.df=Inf
#if having a prior, set prior.mean=c(....), prior.scale=c(.....)
#length of prior mean and prior scale should be equal to the number of predictors
if(! use_prior){
    model = bayesglm(formula, data=use_data, family=binomial(link='logit'), prior.mean=apply(x, 2, mean), prior.scale=Inf, scale=FALSE)
}
if(use_prior){
    model = bayesglm(formula, data=use_data, family=binomial(link='logit'), 
                    prior.mean=prior_mean,
                    prior.scale=prior_scale,
                    prior.mean.for.intercept=prior_mean_int,
                    prior.scale.for.intercept=prior_scale_int,
            scale = FALSE)
}

#oil_palm prediction
class_prediction = 1*(model$fitted.values >= threshold) #if the fitted value is above the threshold, value is changed to binary 1
print(class_prediction)

<br><font face="Helvetica" size="3">Construct confusion matrix, calculate prediction accuracy, posterior CI, print result:</font>

In [ ]:
%%R
#used instead of na.remove to get rid of NA values in 2018 validation dataset
true_label = true_label[!is.na(true_label)]

#generate confusion matrix
bayesian_conf_matrix = matrix(0,2,2)
bayesian_conf_matrix[1,1] = sum(class_prediction + true_label == 0)
bayesian_conf_matrix[2,2] = sum(class_prediction + true_label == 2)
bayesian_conf_matrix[1,2] = sum((class_prediction == 0) & (true_label == 1))
bayesian_conf_matrix[2,1] = sum((class_prediction == 1) & (true_label == 0))
rownames(bayesian_conf_matrix) = c("Predicted non-oil-palm", "Predicted oil-palm")
colnames(bayesian_conf_matrix) = c("Actual non-oil-palm", "Actual oil-palm")
print(bayesian_conf_matrix)

#overall accuracy of model
accu_bayes = sum(class_prediction == true_label) / nrow(data)
print("Overall accuracy:")
print(accu_bayes)

#EDIT: push values to Python and use numpy/matplotlib to display matrix

<br><font face="Helvetica" size="3">Calculate/build posteriors for subsequent runs, print result:</font>

In [ ]:
%%R
# approach posterior distributions of coefficients
# specify number of draws 
num_draw = 2000
post_dist = sim(model, n.sims=num_draw)
coef_matrix = coef(post_dist)

# calculate posterior credible intervals for coefficients
posterior_ci_coef = matrix(NA, ncol(x)+1, 2)
for (i in 1:(ncol(x)+1)){
    posterior_ci_coef[i, ] = unname(quantile(coef_matrix[, i], probs=c(lp, up), na.rm=TRUE))
}

# calculate posterior credible intervals for every data point
posterior_ci_data = matrix(NA, nrow(x), 2)
for(i in 1:nrow(x)){
    temp = as.numeric()
    for(j in 1:num_draw){
        temp[j] = 1 / (1 + exp(-coef_matrix[j, 1] - sum(coef_matrix[j, 2:(length(index)+1)] * x[i, ])))
    }
    posterior_ci_data[i, ] = unname(quantile(temp, probs=c(lp, up), na.rm=TRUE))
}

# build posterior objects for next run
posterior_mean = model$coefficients
posterior_scale = apply(coef(post_dist), 2, sd)

#print the posteriors
posterior_mean

<hr><br>
<font face="Helvetica" size="5"><b>APPLY</b></font>
<br><br>
<font face="Helvetica" size="3">Apply model fits to calculate OP3 for the area covered by the data stack. OP3 = oil palm probabili.</font>

<br><font face="Helvetica" size="3">Generate dummy:</font>

In [ ]:
%%R
#generate dummy for Docker
    #only use env variables if creation fails (will normally return NULL but still create object)
# Sys.setenv(PROJ_LIB="/usr/bin/proj/")
# Sys.getenv("PROJ_LIB")
gdal_translate(src_dataset=in_stack, dst_dataset=in_dummy, of="ENVI", b=1)

<br><font face="Helvetica" size="3">Apply model to stack, execute predictive analysis:</font>

In [ ]:
%%R
#determine bands to use
modelBands = rep(NA, length(stack_names))
for(i in 1:length(modelBands)){
    w = which(bandNames == stack_names[i])
    modelBands[i] = w
}

#create GIS objects
gdalObjStack = new("GDALDataset", in_stack)
gdalObjDummy = new("GDALDataset", in_dummy)
rasterWidth = ncol(gdalObjStack)
rasterRows = nrow(gdalObjStack)

#calculate prediction for each pixel and save
print("Checking a few values...")
for(i in 1:rasterRows){
    oneRasterLine = getRasterData(gdalObjStack, offset=c(i-1,0), region.dim=c(1, rasterWidth))
    hhBand = which(bandNames == "alos2_hh")
    pred = rep(-9999, rasterWidth)
    for(j in 1:rasterWidth){
        #hh = (20*log10(oneRasterLine[j, 1, hhBand])) -83
        hh = oneRasterLine[j, 1, hhBand] #gets hh value at each of the pixels
        #open water mask
        #if(is.na(hh) | hh < -20){ 
        #pred[j] = 0
        #} 
        #else{
            #select bands
            selectBands = oneRasterLine[j, 1, modelBands]
            z = (intercept + sum(posteriors * selectBands))
            pred[j] = exp(z)/(1+ exp(z))
            #z = (intercept + sum(posteriors * scaledBands))
            #pred[j] = exp(z)/(1+ exp(z))
            if ((i/100 == i%/%100) & j == 1000) print(z) #reality check on the model fits
        #}
    }
    #write one row to file
    putRasterData(gdalObjDummy, pred, offset=c(i-1, 0)) #place predicted line in raster into dummy
}
saveDataset(gdalObjDummy, out_pred)

<br><font face="Helvetica" size="3">Convert to GeoTiff:</font>

In [ ]:
%%R
gdal_translate(src_dataset=out_pred, dst_dataset=out_tif, of="GTiff")